# Autoevaluación: Calidad de datos aplicada al proyecto
## Introducción
    ¿Qué encontrará? 
    En esta guía vas a encontrar algunas preguntas que ayudan a orientar un correcto análisis preliminar de calidad de datos.

    ¿Qué construirá?
    En este tutorial realizarás un análisis preliminar de la calidad de los datos utilizando PySpark

    ¿Para qué?
    Adelantar entendimiento de datos del proyecto y validar la estrategia definida para el trabajo en equipo
    
    ¿Qué necesita?
    Los siguientes requisitos se encuentran instalados en la máquina virtual asignada a cada estudiante, específicamente en el ambiente de anaconda llamado "base". Recuerde que tiene a su disposición el tutorial de conexión a máquinas virtuales en la semana 1 de Coursera
    1. Python 3 con pip instalado
    2. Jupyter Labs
    3. Paquetes: Pyspark (3.2.1), pandas (1.2.1), numpy (1.20.0) y matplotlib (3.3.4)
    Otros:
    1. Controlador Connector J(ya se encuentra configurado)
    2. Acceso a servidor remoto MySQL con base de datos relacional "WWImportersTransactional". Recuerde que tiene a su disposición el tutorial de conexión remoto a Mysql en la semana 1 de Coursera

## Intrucciones
A continuación vas a encontrar unas preguntas que ayudarán a guiar un análisis preliminar de calidad de datos. Al final de cada pregunta reflexiona sobre la relación que existe entre esa pregunta y alguna de las dimensiones de calidad definidas en el curso.

El enunciado completo del entendimiento de datos del proyecto lo encuentran en la siguiente semana. Por lo pronto, Infraestructura Visible le comenta que los datos compartidos para el proyecto son registros de vuelos, aeropuertos, información de centros poblados (se asume que un aeropuerto está ubicado en un centro poblado), información de proyección de población y de PIB (Producto Interno Bruto) por departamentos.
Adicionalmente, le comparte una serie de análisis que le gustaría hacer con las fuentes compartidas:
1.	Análisis 1: ¿Cuántos pasajeros salieron de Medellín a Cali durante temporadas altas? (e.g., Semana santa). En general quiere aplicar este análisis a diferentes ciudades de Colombia.
2.	Análisis 2: ¿Qué departamento de Colombia tiene mayor número de sillas salientes desaprovechadas en 2015?
3.	Análisis 3: ¿Qué aeropuerto representa mayor exportación internacional en términos de carga?
4.	Análisis 4: Top 3 de aeropuertos nacionales que podrían reducir su presupuesto en aviones debido a que no transportan un mínimo del 50% de la capacidad que tienen en carga
5.	Análisis 5: ¿Qué áreas de Colombia no tienen cobertura?, ¿cuáles serían los puntos estratégicos para construir aeropuertos con el fin de lograr la cobertura a dichas áreas?


## **Recursos requeridos**

***Datos suministrados***

Los datos los puedes encontrar en la base de datos: ProyectoTransaccional del servidor que manejamos en los tutoriales. También puede encontrar el diccionario de los mismos [aquí](https://github.com/MISW-4402-Analisis-y-Modelado-de-datos/guias/blob/main/docs/Proyecto/Diccionario%20IV.xlsx), ambos recursos requeridos para el desarrollo de esta tarea. Las tablas compartidas son copias de las tablas de los sistemas transaccionales. 

Con el fin de aportar en la organización del trabajo al interior de los grupos, se sugiere que uno de los estudiantes del grupo se encargue de las tablas divipolaCopia y pibCopia. Los otros 3 estudiantes pueden escoger cualquiera de las otras 3 tablas.
-	aeropuertosCopia
-	divipolaCopia
-	vuelosCopia
-	pibCopia
-	proyeccionesCopia

***Tecnología***

Recuerda que está el tutorial de “Entendimiento de datos”, que será de utilidad para el uso de la tecnología utilizada en esta tarea.
- JupyterLab
- Pyspark
## Análisis de calidad de datos

La calidad de los datos consiste en validar la idoneidad de los datos analizando varias dimensiones, entre las cuales resaltamos:

- Completitud: Datos que no existen o no se conocen.
- Unicidad (Duplicidad): Datos que son idénticos en diferentes registros o registros con valores idénticos en atributos en los que no debería ocurrir
- Consistencia: Esta medida se define por la correctitud o integridad del dato, en la definición de su estructura, al interior de una fila o entre diferentes filas de la misma fuente o de diferentes fuentes. El manejo de unidades en los datos y el significado del dato son generadores de inconsistencias.
- Validez: a nivel de formato y de sentido de los datos más alla de sus valores

Te invito a revisar los recursos de calidad de datos que hemos visto en el curso.


## Configuración e importe de paquetes

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
import pyspark.sql.functions as f
import os 
from datetime import datetime
from pyspark.sql import types as t
from pandas_profiling import ProfileReport
#import matplotlib.pyplot as plt
import numpy as np

In [ ]:
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [ ]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

In [ ]:
# Si quiere practicar la conexion con el servidor de base de datos:
db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'
# El usuario es su estudiante _i asignado y su contraseña la encontrará en el archivo excel de Coursera 
db_user = ''
db_psswd = ''

PATH='./'

#### Conexión a fuente de datos
A continuación encuentra las funciones para conectarse a la fuente de datos (archivo csv o base de datos) y retornar un dataframe que es el que se utilizará posteriormente para manipular los datos.

In [ ]:
def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

### Ejemplo de cómo encontrará las respuestas al final
    En cada una de las preguntas encontrará una función dónde deberá modificar el método para retornar el valor especificado en los comentarios dados antes de la definición de la función.

In [2]:
#(Explicación) Función que tiene como objetivo devolver la cantidad de horas del día 
#(La respuesta que se espera) Return: Un número. Ejemplo:5000
#(Extras) No cambiar el nombre de las funcion.
def EjemploFuncionResultadoDelEstudiante():
    # Inserte acá su código para encontrar la cantidad de horas del día 
    return 24

    Al final de este documento encontrára unos "asserts" que utilizarán la función que usted creó y comparará con la respuesta correcta. En caso de que sea acertado obtendrá el siguiene mensaje.

In [3]:
respuestaReal=24
assert EjemploFuncionResultadoDelEstudiante()==respuestaReal, "Tiene un error, en la prueba de ejemplo se esperaba "+str(respuesta)+" y se obtuvo "+ ejemplo()
print("Salió bien la prueba de ejemplo de los datos de Ejemplo")

Salió bien la prueba de ejemplo de los datos de Ejemplo


    En caso de que sea incorrecto saldría un error, como se presenta a continuación. 

In [6]:
respuesta=23
assert EjemploFuncionResultadoDelEstudiante()==respuesta, "Tiene un error, en la prueba de ejemplo se esperaba "+str(EjemploFuncionResultadoDelEstudiante())+" y se obtuvo "+ str(respuesta)
print("Salió bien la prueba de ejemplo de los datos de Ejemplo")

AssertionError: Tiene un error, en la prueba de ejemplo se esperaba 24 y se obtuvo 23

# Nota: esta actividad no es calificable. Sin embargo, le permite avanzar en el entregable de entendimiento de datos.

### Fuente de datos PIB
    1. ¿Cuantos duplicados totales y lógicos(en un subconjunto de columnas en las cuales no debería ocurrir), hay en la fuente de datos de PIB?
    2. ¿Cuáles años diferentes hay la fuente de datos de PIB?
    A continuación desarrolle las funciones para poder dar respuesta a las anteriores preguntas sobre la fuente de datos PIB

In [7]:
#Función que tiene como objetivo devolver la cantidad de duplicados lógicos que tienen la fuente de datos PIB
# Return: Un número. Ejemplo:5000
# No cambiar el nombre de las funcion.
def duplicadoslogicos():
    # Inserte acá su código para encontrar la cantidad de duplicadod lógicos que tiene la fuente de datos PIB 
    return 5002

In [ ]:
#Función que tiene como objetivo devolver la cantidad de duplicados totales que tienen la fuente de datos PIB
# Return: Un número. Ejemplo:5000
# No cambiar el nombre de las funcion.
def duplicadosTotales():
    # Inserte acá su código para encontrar la cantidad de duplicados totales que tiene la fuente de datos PIB 
    return 5001

In [ ]:
#Función que tiene como objetivo devolver los años diferentes que tienen la fuente de datos PIB
# Return: Array de números. Ejemplo:[2013,2014,2015]
# No cambiar el nombre de las funcion.
def aniosEnPIB():
    # Inserte acá su código para encontrar la cantidad de años diferentes que tiene la fuente de datos PIB 
    return [2013,2014,2015]

### Fuente de datos Proyecciones
    3. ¿Cuantos hombres hay por departamento para el año 2014 en la fuente de datos de proyecciones?
    4. ¿Cuales son los 2 valores máximos de la fuente de datos de proyecciones?
    A continuación desarrolle las funciones para poder dar respuesta a las anteriores preguntas sobre la fuente de datos proyecciones

In [ ]:
#Función que tiene como objetivo devolver la cantidad de hombres en el 2014 que tiene la fuente de datos Proyecciones
# Return: 
#         Diccionario con llave el nombre de la columna departamento y valor el número de hombres que tiene en el año 2014. 
#         Ejemplo:{"Antioquia":2300,"Antioquita":5899,"Santander":12344}
# No cambiar el nombre de las funcion.
def hombresDepart2014():
    # Inserte acá su código para encontrar la cantidad de hombres por departamento que tienen en el año 2014
    return {"Antioquia":2300,"Antioquita":5899,"Santander":12344}
    

In [ ]:
#Función que tiene como objetivo devolver los 2 valores máximos que tienen la fuente de datos Proyecciones en la columna ""
# Return: Array de 2 números. Ejemplo:[25000,5000000]
# No cambiar el nombre de las funcion.
def valoresMaximos():
    # Inserte acá su código para encontrar los valores máximos de la fuente de datos proyecciones en la columna ""
    return [25000,5000000]

### Fuente de datos Vuelos
    ¿Cuantos duplicados totales y lógicos hay en la fuente de datos Vuelos?
    ¿Cuantos formatos diferentes de fecha hay en la fuente de datos vuelos?

In [ ]:
#Función que tiene como objetivo devolver la cantidad de duplicados lógicos que tienen la fuente de datos vuelos
# Return: Un número. Ejemplo:5000
# No cambiar el nombre de las funcion.
def duplicadoslogicosVuelos():
    # Inserte acá su código para encontrar la cantidad de duplicadod lógicos que tiene la fuente de datos vuelos
    return 5003

In [ ]:
#Función que tiene como objetivo devolver la cantidad de duplicados totales que tienen la fuente de datos Vuelos
# Return: Un número. Ejemplo:5000
# No cambiar el nombre de las funcion.
def duplicadosTotalesVuelos():
    # Inserte acá su código para encontrar la cantidad de duplicados totales que tiene la fuente de datos vuelos 
    return 5004

In [ ]:
#Función que tiene como objetivo devolver la cantidad de formatos diferentes de fechas que tienen la fuente de datos Vuelos
# Return: Un número. Ejemplo:123
# No cambiar el nombre de las funcion.
def formatosFecha():
    # Inserte acá su código para encontrar la cantidad de formatos de fechas diferentes que tiene la fuente de datos vuelos 
    return 5005

### Fuente de datos Aeropuertos
    ¿Cuáles columnas constantes hay en la fuente de datos Aeropuertos?
    ¿Cuantos valores vacios hay en todas las columnas de la fuente de datos aeropuertos? 

In [ ]:
#Función que tiene como objetivo devolver los nombres de las columnas cosntantes que tienen la fuente de datos Aeropuertos
# Return: Array de strings. Ejemplo:["columna1","columna2","columna3"] donde los nombres deben estar en minúscula
# No cambiar el nombre de las funcion.
def columnasConstantesAeropuertos():
    # Inserte acá su código para encontrar los nombres de las columnas constantes que tiene la fuente de datos Aeropuertos
    return ["columna1","columna2","columna3"]

In [ ]:
#Función que tiene como objetivo devolver la cantidad de vacíos que tienen las columnas en la fuente de datos Aeropuertos
# Return: 
#         Diccionario con llave el nombre de la columna y valor el número de vacíos. 
#         Ejemplo:{"Columna1":2300,"Columna2":5899,"Columna":12344}
# No cambiar el nombre de las funcion.
def vaciosAeropuertos():
    # Inserte acá su código para encontrar la cantidad de vacíos que tienen las columnas en la fuente de datos Aeropuertos
    return {"columna1":2300,"columna2":5899,"columna":12345}

### Fuente de datos Divipola
    ¿Cuales columnas Constantes hay en la fuente de datos Divipola?
    ¿Cuantos nombres de departamento hay? 

In [ ]:
#Función que tiene como objetivo devolver los nombres de las columnas cosntantes que tienen la fuente de datos Divipola
# Return: Array de strings. Ejemplo:["Columna1","Columna2","Columna3"]
# No cambiar el nombre de las funcion.
def columnasConstantesDivipola():
    # Inserte acá su código para encontrar los nombres de las columnas constantes que tiene la fuente de datos Divipola
    return ["Columna1","Columna2","Columna5"]

In [ ]:
# Función que tiene como objetivo devolver la cantidad de departamentos que tienen la fuente de datos Divipola
# Return: Un número. Ejemplo:123
# No cambiar el nombre de las funcion.
def departamentosDivipola():
    # Inserte acá su código para encontrar la cantidad de departamentos diferentes que tiene la fuente de datos Divipola 
    return 5006

### Prueba de las funciones y respuestas:

In [8]:
rDuplicadosLogicos=0
assert duplicadoslogicos()==rDuplicadosLogicos, "Error en la prueba de duplicados lógicos se esperaban " + str(rDuplicadosLogicos) + " se obtuvo " + str(duplicadoslogicos())
print("Salió bien la prueba de duplicados lógicos de los datos PIB")

AssertionError: Error en la prueba de duplicados lógicos se esperaban 0 se obtuvo 5002

In [ ]:
rDuplicadosTotales=10
assert duplicadosTotales()==rDuplicadosTotales, "Error en la prueba de duplicados totales se esperaban " + str(rDuplicadosTotales) + " se obtuvo " + str(duplicadosTotales())
print("Salió bien la prueba de duplicados totales de los datos PIB")

In [ ]:
# Función que tiene como objetivo verificar la respuesta dada por el estudiante con la respuesta correcta respecto a los años en la fuente PIB
# Return: Un boolean. Ejemplo true si tiene todas las respuestas, false de lo contrario. 
# No cambiar el nombre ni tocar las funcion.
rAniosEnPIB=['2006', '2007','2008', '2010', '2011', '2012', '2013', '2014']
def comparar(aniosRespuesta,aniosRespuestaEstudiante):
    if len(aniosRespuesta)!=len(aniosRespuestaEstudiante):
        return False
    for i in aniosRespuesta:
        if not i in aniosRespuestaEstudiante:
            return False
        
    return True

assert comparar(rAniosEnPIB,aniosEnPIB()), "Error en la prueba de años en PIB se esperaban " +str(rAniosEnPIB)+ " se obtuvo " +str(aniosEnPIB())
print("Salió bien la prueba de años de los datos PIB")

In [ ]:
rHombresDepart2014={'antioquia': 2930948, 'atlantico': 1153029, 'bogota, d.c.': 3476538, 'bolivar': 982494,
                    'boyaca': 584185, 'caldas': 477526, 'caqueta': 202486, 'cauca': 698313,
                    'cesar': 544408, 'cordoba': 851746, 'cundinamarca': 1211933, 'choco': 246978,
                    'huila': 524405, 'la guajira': 385429, 'magdalena': 628812, 'meta': 492738,
                    'nariño': 787396, 'norte de santander': 692774, 'quindio': 253680, 'risaralda': 444006,
                    'santander': 1018556, 'sucre': 433385, 'tolima': 654966, 'valle del cauca': 2110069,
                    'arauca': 121163, 'casanare': 197737, 'putumayo': 163476, 'archipielago de san andres': 29938,
                    'amazonas': 37441, 'guainia': 22255, 'guaviare': 41413, 'vaupes': 19342, 'vichada': 52935}
# Función que tiene como objetivo verificar la respuesta dada por el estudiante con la respuesta correcta respecto a los hombres en los departamentos
# Return: Un boolean. Ejemplo true si tiene todas las respuestas, false de lo contrario. 
# No cambiar el nombre ni tocar las funcion.
def compararDiccionario(rHombresDepart2014,eHombresDepart2014):
    if len(rHombresDepart2014)!=len(eHombresDepart2014):
        return False
    for i in rHombresDepart2014.keys():
        if not rHombresDepart2014[i]==eHombresDepart2014[i]:
            return False
    return True
assert compararDiccionario(rHombresDepart2014,hombresDepart2014()), "Error en la prueba de años en Proyecciones se esperaban " +str(rHombresDepart2016)+ " se obtuvo " +str(hombresDepart2016())
print("Salió bien la prueba de años de los datos Proyecciones")

In [ ]:
rValoresMaximos=[55000000, 3476538]
# Función que tiene como objetivo verificar la respuesta dada por el estudiante con la respuesta correcta con respecto a los valores máximos en proyecciones
# Return: Un boolean. Ejemplo true si tiene todas las respuestas, false de lo contrario. 
# No cambiar el nombre ni tocar las funcion.
def compararValoresMaximos(rValoresMaximos,eValoresMaximos):
    if len(rValoresMaximos)!=len(eValoresMaximos):
        return False
    for i in rValoresMaximos:
        if not i in eValoresMaximos:
            return False
        
    return True

assert compararValoresMaximos(rValoresMaximos,valoresMaximos()), "Error en la prueba de valores maximos en Proyecciones se esperaban " +str(rValoresMaximos)+ " se obtuvo " +str(valoresMaximos())
print("Salió bien la prueba de valores maximos en Proyecciones")

In [ ]:
rDuplicadosTotalesVuelos=51678
assert duplicadosTotalesVuelos()==rDuplicadosTotalesVuelos, "Error en la prueba de duplicados totales en vuelos se esperaban " + str(rDuplicadosTotalesVuelos) + " se obtuvo " + str(duplicadosTotalesVuelos())
print("Salió bien la prueba de duplicados totales en vuelos")

In [ ]:
rDuplicadosLogicosVuelos=250278
assert duplicadoslogicosVuelos()==rDuplicadosLogicosVuelos, "Error en la prueba de duplicados lógicos en vuelos se esperaban " + str(rDuplicadosLogicosVuelos) + " se obtuvo " + str(duplicadoslogicosVuelos())
print("Salió bien la prueba de duplicados lógicos en vuelos")

In [ ]:
rFormatosFecha=2
assert formatosFecha()==rFormatosFecha, "Error en la prueba de formato de fecha de vuelos se esperaban " + str(rFormatosFecha) + " se obtuvo " + str(formatosFecha())
print("Salió bien la prueba de formato de fecha de vuelo")

In [ ]:
rColConstantesAeropuertos=['categoria','latitud','longitud_pista','ancho_pista','orientacion']
# Función que tiene como objetivo verificar la respuesta dada por el estudiante con la respuesta correcta de las columnas constantes en Aeropuertos
# Return: Un boolean. Ejemplo true si tiene todas las respuestas, false de lo contrario. 
# No cambiar el nombre ni tocar las funcion.
def compararColConstantesAeropuertos(rColConstantesAeropuertos,eColConstantesAeropuerto):
    if len(rColConstantesAeropuertos)!=len(eColConstantesAeropuerto):
        return False
    for i in rColConstantesAeropuertos:
        if not i in eColConstantesAeropuerto:
            return False
        
    return True

assert compararColConstantesAeropuertos(rColConstantesAeropuertos,columnasConstantesAeropuertos()), "Error en la prueba de valores constantes en Aeropuertos se esperaban " +str(rColConstantesAeropuertos)+ " se obtuvo " +str(columnasConstantesAeropuertos())
print("Salió bien la prueba de valores constantes en Aeropuertos")

In [ ]:
rVaciosAeropuertos={'sigla': 0, 'iata': 482, 'nombre': 0, 'municipio': 0,
                    'departamento': 0, 'categoria': 41, 'latitud': 0, 'longitud': 140,
                    'propietario': 141, 'explotador': 141, 'longitud_pista': 0, 'ancho_pista': 0,
                    'pbmo': 194, 'orientacion': 547, 'elevacion': 141, 'resolucion': 147,
                    'fecha_construccion': 383, 'fecha_vigencia': 460, 'clase': 141, 'tipo': 141,
                    'numero_vuelos_origen': 0, 'gcd_departamento': 0, 'gcd_municipio': 0}
# Función que tiene como objetivo verificar la respuesta dada por el estudiante con la respuesta correcta con respecto a los vacíos de la fuente aeropuertos
# Return: Un boolean. Ejemplo true si tiene todas las respuestas, false de lo contrario. 
# No cambiar el nombre ni tocar las funcion.
def compararDiccionarioVaciosAeropuertos(rVaciosAeropuertos,eVaciosAeropuertos):
    if len(rVaciosAeropuertos)!=len(eVaciosAeropuertos):
        return False
    for i in rVaciosAeropuertos.keys():
        if not rVaciosAeropuertos[i]==eVaciosAeropuertos[i]:
            return False
    return True
assert compararDiccionarioVaciosAeropuertos(rVaciosAeropuertos,vaciosAeropuertos()), "Error en la prueba de vacíos en Aeropuertos se esperaban " +str(rVaciosAeropuertos)+ " se obtuvo " +str(vaciosAeropuertos())
print("Salió bien la prueba de vacíos en Aeropuertos")

In [ ]:
rColConstantesDivipola=['Código Departamento','Nombre Municipio']
# Función que tiene como objetivo verificar la respuesta dada por el estudiante con la respuesta correcta de los valores constantes en la fuente Divipola
# Return: Un boolean. Ejemplo true si tiene todas las respuestas, false de lo contrario. 
# No cambiar el nombre ni tocar las funcion.
def compararColConstantesDivipola(rColConstantesDivipola,eColConstantesDivipola):
    if len(rColConstantesDivipola)!=len(eColConstantesDivipola):
        return False
    for i in rColConstantesDivipola:
        if not i in eColConstantesDivipola:
            return False
        
    return True

assert compararColConstantesDivipola(rColConstantesDivipola,columnasConstantesDivipola()), "Error en la prueba de valores constantes en Divipola se esperaban " +str(rColConstantesDivipola)+ " se obtuvo " +str(columnasConstantesDivipola())
print("Salió bien la prueba de valores constantes en Divipola")

In [ ]:
rdepartamentosDivipola=64
assert departamentosDivipola()==rdepartamentosDivipola, "Error en la prueba de departamentos de la fuente divipola se esperaban " + str(rdepartamentosDivipola) + " se obtuvo " + str(departamentosDivipola())
print("Salió bien la prueba de departamentos de la fuente divipola")